In [1]:
#git clone https://github.com/remaplab/RecSys_Course_AT_PoliMi.git

Cloning into 'RecSys_Course_AT_PoliMi'...
remote: Enumerating objects: 1672, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 1672 (delta 221), reused 209 (delta 209), pack-reused 1428 (from 3)
Receiving objects: 100% (1672/1672), 53.49 MiB | 23.56 MiB/s, done.
Resolving deltas: 100% (972/972), done.


In [2]:
#rm /kaggle/working/RecSys_Course_AT_PoliMi/Recommenders/MatrixFactorization/PureSVDRecommender.py

In [3]:
#cp -a /kaggle/input/recsys-repo-resources/PureSVDRecommender.py /kaggle/working/RecSys_Course_AT_PoliMi/Recommenders/MatrixFactorization/

In [4]:
#cd /kaggle/working/RecSys_Course_AT_PoliMi/

/kaggle/working/RecSys_Course_AT_PoliMi


In [5]:
import json
import pandas as pd
import scipy as sp
import numpy as np
np.int = np.int32
np.float = np.float64
np.bool = np.bool_

In [6]:
from src.Evaluation.Evaluator import EvaluatorHoldout


urm_test=sp.sparse.load_npz("/splitted_urms/urm_test.npz")
urm_train=sp.sparse.load_npz("/splitted_urms/urm_train.npz")
urm_validation=sp.sparse.load_npz("/splitted_urms/urm_validation.npz")

evaluator = EvaluatorHoldout(urm_validation, cutoff_list=[10])

In [ ]:

from fairness_serendipity_fns import compute_fairness, compute_serendipity, compute_recommendations_dict,compute_recommendations_dict,compute_item_popularity
import json

with open('config_utils.json','r') as f:
    data=json.load(f)
    configurations_dict=data['weights_config']
    num_configs=data['num_configs']


In [ ]:
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

evaluator = EvaluatorHoldout(urm_validation, cutoff_list=[10])

item_popularity=compute_item_popularity(urm_train)

def objective(trial):
    topk=trial.suggest_int('topk',15,500,log=True)
    alpha=trial.suggest_float('alpha',0.05,2,log=True)
    recommender=P3alphaRecommender(urm_train)
    recommender.fit(topK=topk,alpha=alpha,implicit=True,normalize_similarity=False)
    result_df, _ = evaluator.evaluateRecommender(recommender)
    return result_df.loc[10, "HIT_RATE"]

def multi_objective(trial):
    topk=trial.suggest_int('topk',15,500,log=True)
    alpha=trial.suggest_float('alpha',0.05,2,log=True)
    recommender=P3alphaRecommender(urm_train)
    recommender.fit(topK=topk,alpha=alpha,implicit=True,normalize_similarity=False)
    rec_dict=compute_recommendations_dict(recommender, urm_validation)
    result_df, _ = evaluator.evaluateRecommender(recommender)
    fairness=compute_fairness(rec_dict, item_popularity, G=4)
    serendipity=compute_serendipity(rec_dict, item_popularity)
    composed_metrics=weights_list[0]*result_df.loc[10, "HIT_RATE"] +  weights_list[1]*result_df.loc[10,"DIVERSITY_GINI"] +weights_list[2]*result_df.loc[10, "NOVELTY"]+ weights_list[3]*serendipity+weights_list[4]*fairness
    return composed_metrics

In [10]:
import optuna
from optuna.samplers import TPESampler

for i in range(1,num_configs+1):
    config=f"c{i}"
    weights_list=configurations_dict[config]
    database_path=f"/kaggle/working/p3Alpha_multi_{config}.db"
    sampler=TPESampler(n_startup_trials=6)
    study=optuna.create_study(study_name=f"p3Alph_multi_hyper_param_tuning_{config}",sampler=sampler,direction="maximize",storage=f'sqlite:///{database_path}',load_if_exists=True)
    
    study.optimize(multi_objective, n_trials=50)


[I 2025-11-16 12:49:52,785] A new study created in RDB with name: p3Alph_multi_hyper_param_tuning_c1


P3alphaRecommender: Similarity column 3043 (100.0%), 2474.52 column/sec. Elapsed time 1.23 sec
EvaluatorHoldout: Processed 6022 (100.0%) in 3.05 sec. Users per second: 1976


[I 2025-11-16 12:49:59,632] Trial 0 finished with value: 0.2975761545179942 and parameters: {'topk': 90, 'alpha': 0.060598821801843186}. Best is trial 0 with value: 0.2975761545179942.


P3alphaRecommender: Similarity column 3043 (100.0%), 2064.49 column/sec. Elapsed time 1.47 sec
EvaluatorHoldout: Processed 6022 (100.0%) in 3.09 sec. Users per second: 1950


[I 2025-11-16 12:50:06,843] Trial 1 finished with value: 0.32711750195775824 and parameters: {'topk': 143, 'alpha': 1.051532488698647}. Best is trial 1 with value: 0.32711750195775824.


...

[I 2025-11-16 13:12:51,649] Trial 47 finished with value: 0.36657349972072667 and parameters: {'topk': 166, 'alpha': 1.5947878074209665}. Best is trial 42 with value: 0.3882329574794748.


P3alphaRecommender: Similarity column 3043 (100.0%), 2224.72 column/sec. Elapsed time 1.37 sec
EvaluatorHoldout: Processed 6022 (100.0%) in 3.05 sec. Users per second: 1973


[I 2025-11-16 13:12:58,682] Trial 48 finished with value: 0.34555583746415525 and parameters: {'topk': 129, 'alpha': 0.06809799004831321}. Best is trial 42 with value: 0.3882329574794748.


P3alphaRecommender: Similarity column 3043 (100.0%), 1789.83 column/sec. Elapsed time 1.70 sec
EvaluatorHoldout: Processed 6022 (100.0%) in 3.15 sec. Users per second: 1914


[I 2025-11-16 13:13:06,422] Trial 49 finished with value: 0.3873882642882587 and parameters: {'topk': 211, 'alpha': 1.1304632152073575}. Best is trial 42 with value: 0.3882329574794748.
